In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nb_006 import *

In [ ]:
imsq = open_image('/DATA/fastai/testpix.png')
imrect = open_image('/DATA/fastai/rectpix.png')

In [ ]:
imsq.show()
imrect.show()

In [ ]:
imrect.px.shape

In [ ]:
imrect.px[:,4,20]

In [ ]:
img = imrect
pts = tensor([(4,4),(12,12),(20,4),(28,12)]).long()
pts.shape

In [ ]:
_,ax = plt.subplots()
img.show(ax)
ax.scatter(pts[:,0], pts[:,1], s=100, marker='.', c='y')

In [ ]:
def my_rotate(degrees):
    angle = degrees * math.pi / 180
    return [[cos(angle), -sin(angle), 0.],
            [sin(angle),  cos(angle), 0.],
            [0.        ,  0.        , 1.]]

m = my_rotate(30)
m = tensor(m)


In [ ]:
img = imrect.clone()
h, w = img.shape
new_size = (h*15,w*15)
igrid = affine_grid(new_size)
flow = affine_mult(igrid,m,img.shape)
new_px = grid_sample(img.px, c, padding_mode='zeros')

#grid = affine_grid(new_size)
#pts_r  = grid[0,pts[:,0],pts[:,1]]
#pts_r = affine_mult(pts_r,m,imrect.shape)
#pxpts = coords_to_pixels(pts_r[None], new_size)


In [ ]:
img2 = Image(new_px)
_,ax = plt.subplots()
img2.show(ax)
#ax.scatter(pxpts[:,0], pxpts[:,1], s=100, marker='.', c='y')

In [ ]:
img2.min()


# Faces

## Setup

Download the dataset [here](https://download.pytorch.org/tutorial/faces.zip) from the pytoch tutorial on transforms. Unzip it in the data directory, so that data/faces/ contains the images and the csv file.

In [ ]:
PATH = Path('data/faces/')

In [ ]:
img_fns = get_image_files(PATH)
img_fns = [fn for fn in img_fns if str(fn)[-3:] == 'jpg']
len(img_fns)

In [ ]:
import pandas as pd
poses = pd.read_csv(PATH/'face_landmarks.csv')

In [ ]:
pose_pnts = []
imgs = []
for i, fname in enumerate(img_fns):
    img = open_image(fname)
    imgs.append(img)
    coords = tensor(np.array(poses[poses['image_name'] == fname.name].iloc[0][1:], dtype=np.long))
    coords = torch.tensor(coords).view(-1,2)
    pose_pnts.append(coords)
pose_pnts = torch.stack(pose_pnts)

In [ ]:
pts = pose_pnts[-1]
pts.dtype

In [ ]:
#export
def points_to_mask(points, shape):
    mask = torch.zeros(shape,dtype=points.dtype)
    rows,cols = pts[:,0], pts[:,1]
    mask[rows,cols] = 1
    return mask.unsqueeze(0)

def mask_to_points(mask):
    return mask.squeeze(0).nonzero()

In [ ]:
i = 12
_,ax = plt.subplots()
img = imgs[i]
img.show(ax)
pts = pose_pnts[i]
ax.scatter(pts[:,0], pts[:,1], s=30, marker='.', c='y')

In [ ]:
img.shape, pts.max(dim=0,keepdim=True), pts.shape

In [ ]:
def pixel_lookup_grid(size):
    size = ((1,)+size)
    N, H, W = size
    grid = FloatTensor(N, H, W, 2)
    linear_points = torch.linspace(0, W, W) if W > 1 else torch.Tensor([-1])
    grid[:, :, :, 0] = torch.ger(torch.ones(H), linear_points).expand_as(grid[:, :, :, 1])
    linear_points = torch.linspace(0, H, H) if H > 1 else torch.Tensor([-1])
    grid[:, :, :, 1] = torch.ger(linear_points, torch.ones(W)).expand_as(grid[:, :, :, 0])
    return grid.permute(0,3,1,2)

def point_sample_nearest(input, coords):
    bs,ch,h,w = input.size()
    sz = torch.tensor([w,h]).float()[None,None]
    coords.add_(1).mul_(sz/2)
    coords = coords[0].round_().long()
    result = input[...,coords[...,1],coords[...,0]]
    return result

def coords_to_pixels(coords, shape):
    h,w = shape
    sz = torch.tensor([w,h]).float()[None,None]
    coords.add_(1).mul_(sz/2)
    coords = coords[0].round_().long()
    return coords[:,(1,0)]

In [ ]:
grid = affine_grid(img.shape)
pts_real = grid[0,pts[:,0],pts[:,1]]
pts_mapped = coords_to_pixels(pts_real[None], img.shape)
#lookup = pixel_lookup_grid(img.shape)
#pts_mapped = point_sample_nearest(lookup, pts_real[None]).long()[0].permute(1,0)
grid.shape, img.shape

In [ ]:
pts_mapped.sort()[0][0:5], pts_mapped.shape
#pts_mapped = pts_mapped.sort()[0]

In [ ]:
pts.sort()[0][0:5], pts.shape

In [ ]:
pts_mapped[0,:], pts[0,:]

In [ ]:
_,ax = plt.subplots()
img.show(ax)
pts = pose_pnts[i]
ax.scatter(pts[:,0], pts[:,1], s=50, marker='.', c='y')
ax.scatter(pts_mapped[:,0], pts_mapped[:,1], s=10, marker='.', c='r')


Let's have a look at the data.

In [ ]:
# def affine_inv_mult(c,m,shape=None):
#     if m is None: return c
#     size = c.size()
#     if shape is not None:
#         h,w = shape
#         m[0,1] *= h/w
#         m[1,0] *= w/h

#     c = c.view(-1,2)
#     a = torch.inverse(m[:2,:2].t()) 
#     c = torch.addmm(-torch.mv(a,m[:2,2]), c, a) 
#     return c.view(size)

def affine_inv_mult(c,m,shape=None):
    if m is None: return c
    size = c.size()
    if shape is not None:
        h,w = shape
        m[0,1] *= h/w
        m[1,0] *= w/h

    c = c.view(-1,2)
    a = torch.inverse(m[:2,:2].t()) 
    c = torch.addmm(-torch.mv(a,m[:2,2]), c, a) 
    return c.view(size)

def filter_oob_coords(c):
    mask = (c[:,:,0] >=-1) * (c[:,:,0] <=1) * (c[:,:,1] >=-1) * (c[:,:,1] <=1)
    return c[mask]

class ImagePoints(ImageBase):
    def __init__(self, pts, shape):
        super().__init__(shape)
        h,w = shape
        self._px_pts = pts.long()
        grid = affine_grid(img.shape)
        self._pts  = grid[0,pts[:,0],pts[:,1]]
    
#     def apply_affine(self, flow, affine_mat, shape):
#         #print('ip:', affine_mat)
#         return affine_mult(flow, affine_mat, shape)
    
#     def affine(self, func, *args, **kwargs):
#         m = func(*args, **kwargs)
#         #print('ip af:', m)
#         #set_trace()
        
#         a = tensor(m)
#         a = torch.inverse(a) 
#         self.affine_mat = a @ self.affine_mat
                    
#         #m = torch.inverse(tensor(m))
#         #self.affine_mat = self.affine_mat @ m
#         return self

    def init_flow(self):
        if self._flow is None:
            self._flow = self._pts.clone()

    def pts_refresh(self):
        if self._affine_mat is not None or self._flow is not None:
            self._pts = self.flow
            #lookup = pixel_lookup_grid(self.shape)
            #pxpts = grid_sample_nearest(lookup, self._pts[None]).long()[0].permute(1,0)
            pxpts = coords_to_pixels(self._pts[None], self.shape)
            self._px_pts = pxpts
            self.sample_kwargs = {}
            #self._flow = None
        return self

    @property
    def pts(self):
        self.pts_refresh()
        return self._px_pts

    def clone(self):
        return self.__class__(self.pts, self.shape)

    @property
    def data(self):
        return self.pts

In [ ]:
def show_pose(img, img_pnts, ax=None):
    if ax is None: _,ax = plt.subplots()
    img.show(ax=ax)
    h,w = img.shape
    pnts = img_pnts.data
    #pnts[:,0] *= -1
    #pnts = (pnts.numpy() + 1) * np.array([w/2,h/2])[None]
    ax.scatter(pnts[:, 0], pnts[:, 1], s=30, marker='.', c='r')

In [ ]:
img = open_image(img_fns[0])
img_pnts = ImagePoints(pose_pnts[0], img.shape)

In [ ]:
show_pose(img, img_pnts)

In [ ]:
@dataclass
class CoordTargetDataset(Dataset):
    x_fns:List[Path]; coords:List[List[float]]
    def __post_init__(self): assert len(self.x_fns)==len(self.coords)
    def __repr__(self): return f'{type(self).__name__} of len {len(self.x_fns)}'
    def __len__(self): return len(self.x_fns)
    def __getitem__(self, i): 
        img = open_image(self.x_fns[i])
        pts = ImagePoints(self.coords[i], img.shape)
        return img, pts

In [ ]:
img_ds = CoordTargetDataset(img_fns, pose_pnts)

In [ ]:
x, y = next(iter(img_ds))
type(x), type(y)

In [ ]:
show_pose(x, y)

## Transforms

In [ ]:
#tfms = [rotate(degrees=(-30,30.))]
#tfms = [rotate(degrees=(20.,21.)), flip_lr(p=0.99)]
tfms = [
    zoom(scale=(1.,2.),row_pct=(0.1,0.9),col_pct=(0.1,0.9)),
    #rotate(degrees=(-80.,80.))
]


In [ ]:
tfm_ds = DatasetTfm(img_ds, tfms, tfm_y=True, padding_mode='zeros', mode='nearest')

In [ ]:
for i in range(5):
    show_pose(*tfm_ds[i])

In [ ]:
x, y = tfm_ds[0]
show_pose(x,y)
mi = y._mask_img
x.shape, y.shape

In [ ]:
mi.shape, mi.show()

In [ ]:
# def show_pose(img, img_pnts, ax=None):
#     if ax is None: _,ax = plt.subplots()
#     img.show(ax=ax)
#     pnts = img_pnts.data
#     h,w = img.shape
#     set_trace()
#     if len(pnts) != 0:
#         #pnts = (pnts.numpy() + 1) * np.array([w/2,h/2])[None]
#         ax.scatter(pnts[0], pnts[1], s=10, marker='.', c='r')
#def show_pose(img, img_pnts, ax=None):
#    if ax is None: _,ax = plt.subplots()
#    img.show(ax=ax)
#    h,w = img.shape
#    pnts = img_pnts.data
#    pnts = (pnts.numpy() + 1) * np.array([w/2,h/2])[None]
#    ax.scatter(pnts[:, 0], pnts[:, 1], s=10, marker='.', c='r')

In [ ]:
_,axs = plt.subplots(1,4, figsize=(30,30))
for i,ax in enumerate(axs): show_pose(*tfm_ds[0], ax=ax)

In [ ]:
%debug

In [ ]:
tfms = [zoom(scale=(1,2),row_pct=(0,1),col_pct=(0,1))]

In [ ]:
tfm_ds = DatasetTfm(img_ds, tfms, tfm_y=True, padding_mode='reflect')

In [ ]:
_,axs = plt.subplots(1,4, figsize=(12,6))
for i,ax in enumerate(axs): show_pose(*tfm_ds[0], ax=ax)

In [ ]:
%debug

In [ ]:
affine_mult??

Affine, coords, crop transform first.

In [ ]:
def apply_pad_coords(c, padding_mode):
    "Apply the padding mode to the coords"
    if padding_mode=='zeros' or padding_mode=='border':
        mask = (c[:,0] >=-1) * (c[:,0] <=1) * (c[:,1] >=-1) * (c[:,1] <=1)
        return c[mask]
    elif padding_mode=='reflect':
        c[c < -1] = -2 - c[c < -1]
        c[c > 1] = 2  - c[c > 1]
    return c

In [ ]:
def affine_inv_mult(c,m):
    "Applies the inverse affine transform described in m"
    size = c.size()
    if shape is not None:
        h,w = shape
        m[0,1] *= h/w
        m[1,0] *= w/h
    c = c.view(-1,2)
    a = torch.inverse(m[:2,:2].t()) 
    c = torch.addmm(-torch.mv(a,m[:2,2]), c, a) 
    return c.view(size)

- Problem in affine: we must apply the inverse tranformation (affine_inv_mult function)
- Problem in coord: we must also apply the inverse transformation (flag invert)
- Probem in crop (and other transforms and the beginning): we need the size of the image where as if the object pass is a list of points, it doesn't have it. To work around this, we create a class that will wrap the list of points and the image size, with a function size (to mimic the behavior of img.size())

In [ ]:
@dataclass 
class TargetCoords():
    coords:Tensor
    img_size:torch.Size
    
    def size(self, int=None) -> Union[int,torch.Size]: 
        if int is None: return self.img_size
        else:           return self.img_size[int]
        
    def clone(self): return TargetCoords(self.coords.clone(), self.img_size)

In [ ]:
tc = TargetCoords(pose_pnts[0], open_image(img_fns[0]).size())

In [ ]:
tc.size(), tc.size(1)

In [ ]:
TfmY = IntEnum('TfmY', 'No Pixel Mask Coord BBox')

class DatasetTfm(Dataset):
    def __init__(self, ds:Dataset, tfms:Collection[Callable]=None, tfm_y:TfmY=TfmY.No, **kwargs):
        self.ds,self.tfms,self.tfm_y,self.kwargs = ds,tfms,tfm_y,kwargs
        
    def __len__(self): return len(self.ds)
    def __getattr__(self, k): return getattr(self.ds, k)
    
    def __getitem__(self,idx):
        if isinstance(idx, tuple): idx,xtra = idx
        else: xtra={}
        x,y = self.ds[idx]
        if self.tfm_y == TfmY.Coord: y = TargetCoords(y, x.size())
        x = apply_tfms(self.tfms, x, **{**self.kwargs, **xtra})
        if self.tfm_y == TfmY.Coord: y = apply_tfms(self.tfms, y, do_resolve=False, is_coord=coord, x_sz=x.size(), 
                                                    **{**self.kwargs, **xtra})
        return apply_tfms(self.tfms, x, **{**self.kwargs, **xtra}), y

In [ ]:
TfmY = IntEnum('TfmY', 'No Pixel Mask Coord BBox')

class TfmDataset(Dataset):
    def __init__(self, ds:Dataset, tfms:Collection[Callable]=None, tfm_y:TfmY=TfmY.No, **kwargs):
        self.ds,self.tfms,self.tfm_y,self.kwargs = ds,tfms,tfm_y,kwargs

    def __len__(self): return len(self.ds)

    def __getitem__(self,idx):
        x,y = self.ds[idx]
        if self.tfms is not None:
            tfm = apply_tfms(self.tfms)
            if self.tfm_y == TfmY.Coord: y = TargetCoords(y, x.size())
            x = apply_tfms(self.tfms, x, **self.kwargs)
            if self.tfm_y != TfmY.No and y is not None:
                coord,seg = (self.tfm_y == TfmY.Coord),(self.tfm_y == TfmY.Mask)
                y = tfm(y, segmentation=seg, is_coord=coord, **self.kwargs).coords
        return x,y

In [ ]:
def get_resize_target(img_sz, crop_target, do_crop=False):
    if crop_target is None: return None
    ch,r,c = img_sz
    target_r,target_c = crop_target
    ratio = (min if do_crop else max)(r/target_r, c/target_c)
    return ch,round(r/ratio),round(c/ratio)

In [ ]:
def _apply_affine(img, size=None, padding_mode='reflect', do_crop=False, aspect=None, mult=32,
                  mats=None, func=None, crop_func=None, is_coord=False, **kwargs):
    if size is not None and not is_listy(size):
        size = listify(size,2) if aspect is None else get_crop_target(size, aspect, mult)
    if (not mats) and func is None and size is None: return img
    resize_target = get_resize_target(img.size(), size, do_crop=do_crop)
    c = img.coords if is_coord else affine_grid(img, torch.eye(3), size=resize_target)
    if func is not None: c = func(c, img.size(), invert=is_coord)
    if mats:
        m = affines_mat(mats)
        c = affine_mult(c, img.new_tensor(m)) if not is_coord else affine_inv_mult(c, img.coords.new_tensor(m))
    if is_coord:
        if resize_target is None: resize_target = img.size()
        res = TargetCoords(apply_pad_coords(c, padding_mode=padding_mode), torch.Size(resize_target))
    else: 
        res = grid_sample(img, c, padding_mode=padding_mode, **kwargs)
        if padding_mode=='zeros': padding_mode='constant'
    if crop_func is not None: res = crop_func(res, size=size, padding_mode=padding_mode, is_coord=is_coord)
    return res

def apply_affine(mats=None, func=None, crop_func=None):
    return partial(_apply_affine, mats=mats, func=func, crop_func=crop_func)

nb_003a.apply_affine = apply_affine

Test on a rotation

In [ ]:
tfms = [rotate(degrees=(-30,30.))]

In [ ]:
tfm_ds = TfmDataset(img_ds, tfms, TfmY.Coord, padding_mode='zeros')

In [ ]:
def show_pose(img, pnts, ax=None):
    if ax is None: _,ax = plt.subplots()
    ax.imshow(img.numpy().transpose(1,2,0))
    w,h = img.size(2),img.size(1)
    if len(pnts) != 0:
        pnts = (pnts.numpy() + 1) * np.array([w/2,h/2])[None]
        ax.scatter(pnts[:, 0], pnts[:, 1], s=10, marker='.', c='r')

In [ ]:
_,axs = plt.subplots(1,4, figsize=(12,6))
for i,ax in enumerate(axs): show_pose(*tfm_ds[0], ax=ax)

Test on a zoom

In [ ]:
tfms = [zoom_tfm(scale=(1,2),row_pct=(0,1),col_pct=(0,1))]

In [ ]:
tfm_ds = TfmDataset(img_ds, tfms, TfmY.Coord, padding_mode='zeros', size=100, do_crop=True)

In [ ]:
_,axs = plt.subplots(1,4, figsize=(12,6))
for i,ax in enumerate(axs): show_pose(*tfm_ds[0], ax=ax)

Whole pipeline: we pass is_coord as an argument to every transform.

In [ ]:
def _apply_tfm_func(pixel_func,lighting_func,affine_func,start_func, x, segmentation=False, is_coord=False, **kwargs):
    if not np.any([pixel_func,lighting_func,affine_func,start_func]): return x
    x = x.clone()
    if start_func is not None:  x = start_func(x, is_coord=is_coord)
    if affine_func is not None: x = affine_func(x, is_coord=is_coord, **kwargs)
    if lighting_func is not None and not segmentation and not is_coord: x = lighting_func(x)
    if pixel_func is not None: x = pixel_func(x, is_coord=is_coord)
    return x

def apply_tfms(tfms):
    resolve_tfms(tfms)
    grouped_tfms = dict_groupby(listify(tfms), lambda o: o.tfm_type)
    start_tfms,affine_tfms,coord_tfms,pixel_tfms,lighting_tfms,crop_tfms = [
        (grouped_tfms.get(o)) for o in TfmType]
    lighting_func = apply_lighting(compose(lighting_tfms))
    mats = [o() for o in listify(affine_tfms)]
    affine_func = apply_affine(mats, func=compose(coord_tfms), crop_func=compose(crop_tfms))
    return partial(_apply_tfm_func,
        compose(pixel_tfms),lighting_func,affine_func,compose(start_tfms))

nb_003a.apply_tfms = apply_tfms

Simple padding

In [ ]:
def pad_coord(x:TargetCoords, row_pad:int, col_pad:int):
    c,h,w = x.size()
    pad = torch.Tensor([w/(w + 2*col_pad), h/(h + 2*row_pad)])
    new_sz = torch.Size([c, h+2*row_pad, w+2*col_pad]) 
    return TargetCoords(x.coords.mul_(pad[None]), new_sz)

In [ ]:
@reg_transform
def pad(x, padding, mode='reflect', is_coord=False) -> TfmType.Start:
    if is_coord: return pad_coord(x, padding, padding)
    else: return F.pad(x[None], (padding,)*4, mode=mode)[0]

In [ ]:
tfms = [pad_tfm(padding=100)]

In [ ]:
tfm_ds = TfmDataset(img_ds, tfms, TfmY.Coord)

In [ ]:
_,axs = plt.subplots(1,4, figsize=(12,6))
for i,ax in enumerate(axs): show_pose(*tfm_ds[0], ax=ax)

Not implemented: the points aren't reflected (is it important?)

Simple crop

In [ ]:
def crop_coords(x:TargetCoords, row:int, col:int, rows:int, cols:int):
    ch,h,w = x.size()
    c = x.coords    
    c.mul_(torch.Tensor([w/cols,h/rows])[None])
    c.add_(-1 + torch.Tensor([w/cols-2*col/cols,h/rows-2*row/rows])[None])
    c = apply_pad_coords(c, padding_mode='zeros')
    new_sz = torch.Size([ch, rows, cols])
    return TargetCoords(c, new_sz)

In [ ]:
@reg_transform
def crop(x, size, is_coord=False, row_pct:uniform=0.5, col_pct:uniform=0.5) -> TfmType.Pixel:
    size = listify(size,2)
    rows,cols = size
    row = int((x.size(1)-rows+1) * row_pct)
    col = int((x.size(2)-cols+1) * col_pct)
    if is_coord: return crop_coords(x,row,col,rows,cols)
    else:        return x[:, row:row+rows, col:col+cols].contiguous()

In [ ]:
tfms = [crop_tfm(size=200,row_pct=(0,1),col_pct=(0,1))]

In [ ]:
tfm_ds = TfmDataset(img_ds, tfms, TfmY.Coord)

In [ ]:
_,axs = plt.subplots(1,4, figsize=(12,6))
for i,ax in enumerate(axs): show_pose(*tfm_ds[0], ax=ax)

Crop pad transform

In [ ]:
@reg_transform
def crop_pad(x, size, padding_mode='reflect', is_coord=False,
             row_pct:uniform = 0.5, col_pct:uniform = 0.5) -> TfmType.Crop:
    size = listify(size,2)
    rows,cols = size
    if x.size(1)<rows or x.size(2)<cols:
        row_pad = max((rows-x.size(1)+1)//2, 0)
        col_pad = max((cols-x.size(2)+1)//2, 0)
        if is_coord: x = pad_coord(x, row_pad, col_pad)
        else: x = F.pad(x[None], (col_pad,col_pad,row_pad,row_pad), mode=padding_mode)[0]
    
    row = int((x.size(1)-rows+1)*row_pct)
    col = int((x.size(2)-cols+1)*col_pct)
    if is_coord: return crop_coords(x,row,col,rows,cols)
    x = x[:, row:row+rows, col:col+cols]
    return x.contiguous() # without this, get NaN later - don't know why

In [ ]:
tfms = [crop_pad_tfm(row_pct=(0,1),col_pct=(0,1))]

In [ ]:
tfm_ds = TfmDataset(img_ds, tfms, TfmY.Coord, padding_mode='zeros', do_crop=True, size=100)

In [ ]:
_,axs = plt.subplots(1,4, figsize=(12,6))
for i,ax in enumerate(axs): show_pose(*tfm_ds[0], ax=ax)

Coord transform (where we see the need for the invert transformation)

In [ ]:
@reg_transform
def tilt(c, img_size, invert=False, direction:rand_int=0, magnitude:uniform=0) -> TfmType.Coord:
    orig_pts = [[-1,-1], [-1,1], [1,-1], [1,1]]
    if direction == 0:   targ_pts = [[-1,-1], [-1,1], [1,-1-magnitude], [1,1+magnitude]]
    elif direction == 1: targ_pts = [[-1,-1-magnitude], [-1,1+magnitude], [1,-1], [1,1]]
    elif direction == 2: targ_pts = [[-1,-1], [-1-magnitude,1], [1,-1], [1+magnitude,1]]
    elif direction == 3: targ_pts = [[-1-magnitude,-1], [-1,1], [1+magnitude,-1], [1,1]]  
    coeffs = find_coeffs(targ_pts, orig_pts) if invert else find_coeffs(orig_pts, targ_pts)
    return apply_perspective(c, coeffs)

In [ ]:
tfms = [tilt_tfm(direction=(0,3), magnitude=(-0.4,0.4))]

In [ ]:
tfm_ds = TfmDataset(img_ds, tfms, TfmY.Coord, padding_mode='zeros', do_crop=True, size=100)

In [ ]:
_,axs = plt.subplots(1,4, figsize=(12,6))
for i,ax in enumerate(axs): show_pose(*tfm_ds[0], ax=ax)

Flip

In [ ]:
def flip_lr_coord(x):
    x.coords[:,0] = -x.coords[:,0]
    return x

In [ ]:
@reg_transform
def flip_lr(x, is_coord=False) -> TfmType.Pixel: 
    if is_coord: return flip_lr_coord(x)
    else:        return x.flip(2)

In [ ]:
tfms = [flip_lr_tfm(p=0.5)]

In [ ]:
tfm_ds = TfmDataset(img_ds, tfms, TfmY.Coord, padding_mode='zeros', do_crop=True, size=100)

In [ ]:
_,axs = plt.subplots(1,4, figsize=(12,6))
for i,ax in enumerate(axs): show_pose(*tfm_ds[0], ax=ax)